In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [17]:
# Importing the dataset
df = pd.read_csv('braznikova2_prep.csv', sep=';')
df.head(10)

,Y,X3,X5,X6,X7,X8,X9,X10,X12,X13,X14
0,0,0,0,659,0,1092.0,13884.0,0,0,0,1
1,0,0,1,1518,1,348.0,11700.0,0,0,1,1
2,0,0,0,476,2,504.0,6786.0,0,0,1,1
3,0,0,0,1310,3,418.0,12948.0,0,0,1,1
4,0,1,2,2029,4,108.0,11427.0,0,0,0,0
5,0,1,3,3585,5,120.0,11856.0,0,1,0,0
6,1,0,4,4621,6,6.0,14508.0,0,0,0,0
7,1,0,5,8150,7,292.0,11310.0,0,0,0,0
8,0,0,0,1624,8,290.0,12324.0,0,0,0,0
9,0,1,6,3790,9,660.0,13455.0,0,0,0,0


In [3]:
# Removing Correlated Data
#df = df.drop(['X5','X3'], axis='columns', inplace=False)
#f.shape

In [4]:
# Splitting the dataset into the Training set and Test set
X = df.iloc[:, 1:]
y = df['Y']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [5]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler().fit(X_train)
X_train = sc_X.transform(X_train)
X_test = sc_X.transform(X_test)

In [6]:
# Baseline model
import statsmodels.api as sm
lr = sm.Logit(y_train, X_train).fit()
print(lr.summary2())

Optimization terminated successfully.
         Current function value: 0.075376
         Iterations 34
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.887     
Dependent Variable: Y                AIC:              28.2914   
Date:               2021-12-04 13:13 BIC:              48.3647   
No. Observations:   55               Log-Likelihood:   -4.1457   
Df Model:           9                LL-Null:          -36.572   
Df Residuals:       45               LLR p-value:      1.5410e-10
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     34.0000                                      
------------------------------------------------------------------
         Coef.    Std.Err.     z     P>|z|     [0.025      0.975] 
------------------------------------------------------------------
x1      -3.0509     2.4797  -1.2303  0.2186     -7.9110     1.8093
x2       1.5377     2.2986   0.6690  0.5035     -2.9675     6

In [7]:
# Automatic Feature Selection
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
estimator=LogisticRegression()
selector=RFECV(estimator, step=1)
selector=selector.fit(X_train, y_train)
print(selector.n_features_)
print(selector.support_)
print(selector.ranking_)

1
[False False  True False False False False False False False]
[ 3  4  1  8  6  7  2 10  9  5]


In [8]:
# Selected Features
selected_columns=[]
for i in range(len(X_train[0])):
    if selector.support_[i]==1:
        selected_columns.append(i)
print(selected_columns)
print('№\tIndex\tFeature')
i=0
for column in selected_columns:
    print(str(i)+'\t'+str(column)+'\t'+str(df.columns[column]))
    i=i+1

[2]
№	Index	Feature
0	2	X5


Одна переменная значима в модели. Однако чтобы имел смысл при построении классификации и кластеризации я отобрала из selector.ranking топ-5 переменных, которые стремятся к значимости по значению p-value. Помимо Х5 - это X10,X3,X6,X14. 

In [9]:
# Leaving only Selected Features
X_train_select=X_train[:, selected_columns]
X_test_select=X_test[:, selected_columns]

In [10]:
# p-value < 1% Features
X_train = X_train[:,[True, True, True, False, False, False, True, False, False, True]]
X_test = X_test[:,[True, True, True, False, False, False, True, False, False, True]]

In [11]:
# Fitting Logistic Regression to the Training set 
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 13).fit(X_train, y_train)

In [12]:
# Predicting the Test set results
y_pred = lr.predict(X_test)
lr.score(X_test,y_test)

0.8571428571428571

Высокое качество модели.  86% наблюедний верно распознаны.

In [14]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[9 1]
 [1 3]]


Неверно распознано 2 обьекта.

In [16]:
import pickle
obj = {'X_train': X_train, 'X_test': X_test,'y_train': y_train,'y_test': y_test}
output = open('prep.pkl', 'wb')
pickle.dump(obj, output, 2)
output.close()